In [1]:
import cv2 as cv
import numpy as np
import onnxruntime as ort
import time 

path_to_frozen_inference_graph = 'frozen_inference_graph.pb'
path_coco_model= 'mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'
net = cv.dnn.readNetFromTensorflow(path_to_frozen_inference_graph,path_coco_model)
labels = ['convertible', 'hatchback', 'heavy-duty', 'sedan', 'sport', 'suv', 'truck', 'van']

cap = cv.VideoCapture('test5.mp4')
_, frame = cap.read()
height, width, _ = frame.shape

session = ort.InferenceSession('model.onnx') 

input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    colors = np.random.randint(125, 255, (80, 3))
    img = cv.resize(frame,(width,height))
    blob = cv.dnn.blobFromImage(img, swapRB=True)

#     time2 = time.time()

#     time3 = time.time()
#     print(f'time3-2: ${time3-time2}')
    net.setInput(blob)
#     time4 = time.time()
#     print(f'time4-3: ${time4-time3}')
    boxes, _ = net.forward(["detection_out_final", "detection_masks"])
#     time5 = time.time()
#     print(f'time5-4: ${time5-time4}')
    detection_count = boxes.shape[2]
#     time6 = time.time()
#     print(f'time6-5: ${time6-time5}')
    for i in range(detection_count):
        box = boxes[0, 0, i]
        class_id = box[1]
        score = box[2]
        if class_id == 2:
            if score < 0.7:
                continue
            x = int(box[3] * width)
            y = int(box[4] * height)
            x2 = int(box[5] * width)
            y2 = int(box[6] * height)
            cv.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)    
            
            to_resize = img[y:y2, x:x2]
            resized = cv.resize(to_resize, (300, 300))
            input_tensor = np.transpose(resized, (2, 0, 1))
            input_tensor = np.expand_dims(input_tensor, axis=0)
            input_tensor = input_tensor.astype(np.float32) 
            
            output = session.run(None, {input_name: input_tensor})
            index = np.argmax(output)
            cv.rectangle(img, (x, y), (x+100, y-30), (155, 155, 155), -1)
            cv.putText(img, labels[index], (x+10, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
#     time7 = time.time()
#     print(f'time7-6: ${time7-time6}')        
    cv.imshow('Frame', img)
    
    if cv.waitKey(1) == ord('q'):
        break


cap.release()
cv.destroyAllWindows()